## Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 9.4 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 4.6 MB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 3.2 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 873 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 68.3 MB/s 
     |████████████████████████████████| 182 kB 75.3 MB/s 


In [ ]:
# !pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html

In [ ]:
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

## Import Packages

In [ ]:
import torch
from torch import nn
from torch_geometric.nn import RGCNConv,FastRGCNConv,GCNConv,GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import json
import os
from datetime import datetime as dt

from transformers import pipeline

from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef

## Global Variables

In [ ]:
root='/content/drive/MyDrive/Twibot-20/'
device='cpu'

## Load Data

In [ ]:

df_train=pd.read_json('/content/drive/MyDrive/Twibot-20/train.json')

df_test=pd.read_json('/content/drive/MyDrive/Twibot-20/test.json')

df_support=pd.read_json('/content/drive/MyDrive/Twibot-20/support.json')

df_dev=pd.read_json('/content/drive/MyDrive/Twibot-20/dev.json')

df_train=df_train.iloc[:,[0,1,2,3,5]]
df_test=df_test.iloc[:,[0,1,2,3,5]]
df_support=df_support.iloc[:,[0,1,2,3]]
df_dev=df_dev.iloc[:,[0,1,2,3,5]]
df_support['label']='None'

df_data_labeled=pd.concat([df_train,df_dev,df_test],ignore_index=True)
df_data=pd.concat([df_train,df_dev,df_test,df_support],ignore_index=True)


## EDA

In [ ]:
df_train.head()

,ID,profile,tweet,neighbor,label
0,17461978,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",[RT @CarnivalCruise: 🎉 Are you ready to see wh...,None,0
1,1297437077403885568,"{'id': '1297437077403885568 ', 'id_str': '1297...",None,"{'following': ['170861207', '23970102', '47293...",1
2,17685258,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",[RT @realDonaldTrump: THANK YOU #RNC2020! http...,"{'following': ['46464108', '21536398', '186434...",0
3,15750898,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",[A family fears they may have been cheated out...,"{'following': ['2324715174', '24030137', '2336...",0
4,1659167666,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",[RT @VonteThePlug: Yeah but he ain’t got one h...,"{'following': ['1628313708', '726405625', '130...",1


In [ ]:
df_train.count()

ID          8278
profile     8278
tweet       8223
neighbor    7524
label       8278
dtype: int64

In [ ]:
df_test.head()

,ID,profile,tweet,neighbor,label
0,1188812492010487808,"{'id': '1188812492010487808 ', 'id_str': '1188...",[RT @clevelanddotcom: Three Ohio House Republi...,None,1
1,155659213,"{'id': '155659213 ', 'id_str': '155659213 ', '...","[We touch our hair 96 times a day on average, ...",None,0
2,147725246,"{'id': '147725246 ', 'id_str': '147725246 ', '...",['He Looked Like He Knew What He Was Doing': C...,"{'following': ['36734275', '20713061', '755419...",0
3,1296248637194895360,"{'id': '1296248637194895360 ', 'id_str': '1296...","[Estamos abiertos a colaboraciones, por lo cuá...","{'following': ['87818409', '41390292', '140910...",1
4,1339835893,"{'id': '1339835893 ', 'id_str': '1339835893 ',...",[The suffragists chose purple and gold to repr...,None,0


In [ ]:
df_test.count()

ID          1183
profile     1183
tweet       1173
neighbor    1074
label       1183
dtype: int64

In [ ]:
df_support.head()

,ID,profile,tweet,neighbor,label
0,1082775333336768512,"{'id': '1082775333336768517 ', 'id_str': '1082...",[RT @RandyRRQuaid: #RNC KICKOFF with Randy’s N...,None,None
1,1076983321438142464,"{'id': '1076983321438142464 ', 'id_str': '1076...",[RT @yogagenie: 02/23/20 ~ @GeneStump1 former...,None,None
2,1166391878264246272,"{'id': '1166391878264246272 ', 'id_str': '1166...","[@joaocaetano aí sim ⚡\n, RT @loud_victor: Mai...",None,None
3,103593224,"{'id': '103593224 ', 'id_str': '103593224 ', '...",[RT @TheDemCoalition: FACT: Seven former Trump...,None,None
4,1274010352683016192,"{'id': '1274010352683016196 ', 'id_str': '1274...",[Man Ellen isn’t even funny\nShe literally smi...,None,None


In [ ]:
df_support.count()

ID          217754
profile     217746
tweet       194297
neighbor      1185
label       217754
dtype: int64

In [ ]:
df_dev.head()

,ID,profile,tweet,neighbor,label
0,1224667050301255680,"{'id': '1224667050301255681 ', 'id_str': '1224...","[@SparklesOnlyme পুরোনো এইদিনের কথা\n, @Barira...","{'following': ['1118754066937462784', '1018358...",0
1,738340405,"{'id': '738340405 ', 'id_str': '738340405 ', '...",[@barstoolbets @betthehorses @JordanByrne70 @t...,None,1
2,108701392,"{'id': '108701392 ', 'id_str': '108701392 ', '...",[On the job hunt? Don’t let an old LinkedIn pr...,None,0
3,10156532,"{'id': '10156532 ', 'id_str': '10156532 ', 'na...",[Too cute! Prince George's official christenin...,"{'following': ['137196549', '108407162', '1629...",1
4,17525171,"{'id': '17525171 ', 'id_str': '17525171 ', 'na...",[RT @GCoxVariety: Got questions about the digi...,None,0


In [ ]:
df_dev.count()

ID          2365
profile     2365
tweet       2350
neighbor    2141
label       2365
dtype: int64

## Preprocessing

In [ ]:
# Load labels

path = root+'label.pt'
if not os.path.isfile(path):
  labels=torch.LongTensor(df_data_labeled['label']).to(device)
  torch.save(labels, path)
else:
  labels = torch.load(root+"label.pt").to(device)

In [ ]:
# Load description and preprocess

path = root+'description.npy'
if not os.path.isfile(path):
  description=[]
  for i in range (df_data.shape[0]):
    if df_data['profile'][i] is None or df_data['profile'][i]['description'] is None:
      description.append('None')
    else:
      description.append(df_data['profile'][i]['description'])
  description=np.array(description)
  np.save(path,description)
else:
  description=np.load(path,allow_pickle=True)

In [ ]:
# Load tweets and preprocess

path = root+'tweets.npy'
if not os.path.isfile(path):
  tweets=[]
  for i in range (df_data.shape[0]):
    one_usr_tweets=[]
    if df_data['tweet'][i] is None:
      one_usr_tweets.append('')
    else:
      for each in df_data['tweet'][i]:
        one_usr_tweets.append(each)
    tweets.append(one_usr_tweets)
  tweets=np.array(tweets)
  np.save(path,tweets)
else:
  tweets=np.load(path,allow_pickle=True)

In [ ]:
# description embedding

path = root+"description_tensor.pt"
if not os.path.isfile(path):
  description=np.load(root+'description.npy',allow_pickle=True)
  feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base",device=0)
  description_vec=[]
  for each in tqdm(description):
    feature=torch.Tensor(feature_extraction(each))
    for (i,tensor) in enumerate(feature[0]):
      if i==0:
        feature_tensor=tensor
      else:
        feature_tensor+=tensor
    feature_tensor/=feature.shape[1]
    description_vec.append(feature_tensor)
  description_tensor=torch.stack(description_vec,0).to(device)
  torch.save(description_tensor,path)
else:
  description_tensor=torch.load(root+"description_tensor.pt").to(device)

In [ ]:
# tweet embedding

path = root+"tweets_tensor.pt"
if not os.path.isfile(path):
  tweets=np.load(root+"/tweets.npy",allow_pickle=True)
  print('Loading RoBerta')
  feature_extract=pipeline('feature-extraction',model='roberta-base',tokenizer='roberta-base',device=0,padding=True, truncation=True,max_length=500, add_special_tokens = True)
  tweets_list=[]
  for each_person_tweets in tqdm(tweets):
    for j,each_tweet in enumerate(each_person_tweets):
      each_tweet_tensor=torch.tensor(feature_extract(each_tweet))
      for k,each_word_tensor in enumerate(each_tweet_tensor[0]):
        if k==0:
          total_word_tensor=each_word_tensor
        else:
          total_word_tensor+=each_word_tensor
      total_word_tensor/=each_tweet_tensor.shape[1]
      if j==0:
        total_each_person_tweets=total_word_tensor
      else:
        total_each_person_tweets+=total_word_tensor
    total_each_person_tweets/=len(each_person_tweets)
    tweets_list.append(total_each_person_tweets)
  tweet_tensor=torch.stack(tweets_list).to(device)
  torch.save(tweet_tensor,path)
else:
  tweets_tensor=torch.load(root+"tweets_tensor.pt").to(device)

In [ ]:
# numerical properties embedding

path0 = root+'numerical_properties_tensor.pt'
if not os.path.isfile(path0):
  path0=root
  if not os.path.isfile(path+"followers_count.pt"):
    
    # followers count
    followers_count=[]
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['followers_count'] is None:
        followers_count.append(0)
      else:
        followers_count.append(df_data['profile'][i]['followers_count'])
    followers_count=torch.tensor(np.array(followers_count,dtype=np.float32)).to(device)
    torch.save(followers_count,path+"followers_count.pt")

    # friends count
    friends_count=[]
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['friends_count'] is None:
        friends_count.append(0)
      else:
        friends_count.append(df_data['profile'][i]['friends_count'])
    friends_count=torch.tensor(np.array(friends_count,dtype=np.float32)).to(device)
    torch.save(friends_count,path+'friends_count.pt')

    # screen name
    screen_name_length=[]
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['screen_name'] is None:
        screen_name_length.append(0)
      else:
        screen_name_length.append(len(df_data['profile'][i]['screen_name']))
    screen_name_length=torch.tensor(np.array(screen_name_length,dtype=np.float32)).to(device)
    torch.save(screen_name_length,path+'screen_name_length.pt')

    # favorites count
    favourites_count=[]
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['favourites_count'] is None:
        favourites_count.append(0)
      else:
        favourites_count.append(df_data['profile'][i]['favourites_count'])
    favourites_count=torch.tensor(np.array(favourites_count,dtype=np.float32)).to(device)
    torch.save(favourites_count,path+'favourites_count.pt')

    # active days
    active_days=[]
    date0=dt.strptime('Tue Sep 1 00:00:00 +0000 2020 ','%a %b %d %X %z %Y ')
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['created_at'] is None:
        active_days.append(0)
      else:
        date=dt.strptime(df_data['profile'][i]['created_at'],'%a %b %d %X %z %Y ')
        active_days.append((date0-date).days)
    active_days=torch.tensor(np.array(active_days,dtype=np.float32)).to(device)
    torch.save(active_days,path+'active_days.pt')

    # status count
    statuses_count=[]
    for i in range (df_data.shape[0]):
      if df_data['profile'][i] is None or df_data['profile'][i]['statuses_count'] is None:
        statuses_count.append(0)
      else:
        statuses_count.append(int(df_data['profile'][i]['statuses_count']))
    statuses_count=torch.tensor(np.array(statuses_count,dtype=np.float32)).to(device)
    torch.save(statuses_count,path+'statuses_count.pt')

  else:
    active_days=torch.load(path+"active_days.pt")
    screen_name_length=torch.load(path+"screen_name_length.pt")
    favourites_count=torch.load(path+"favourites_count.pt")
    followers_count=torch.load(path+"followers_count.pt")
    friends_count=torch.load(path+"friends_count.pt")
    statuses_count=torch.load(path+"statuses_count.pt")

  active_days=pd.Series(active_days.to('cpu').detach().numpy())
  active_days=(active_days-active_days.mean())/active_days.std()
  active_days=torch.tensor(np.array(active_days))

  screen_name_length=pd.Series(screen_name_length.to('cpu').detach().numpy())
  screen_name_length_days=(screen_name_length-screen_name_length.mean())/screen_name_length.std()
  screen_name_length_days=torch.tensor(np.array(screen_name_length_days))

  favourites_count=pd.Series(favourites_count.to('cpu').detach().numpy())
  favourites_count=(favourites_count-favourites_count.mean())/favourites_count.std()
  favourites_count=torch.tensor(np.array(favourites_count))

  followers_count=pd.Series(followers_count.to('cpu').detach().numpy())
  followers_count=(followers_count-followers_count.mean())/followers_count.std()
  followers_count=torch.tensor(np.array(followers_count))

  friends_count=pd.Series(friends_count.to('cpu').detach().numpy())
  friends_count=(friends_count-friends_count.mean())/friends_count.std()
  friends_count=torch.tensor(np.array(friends_count))

  statuses_count=pd.Series(statuses_count.to('cpu').detach().numpy())
  statuses_count=(statuses_count-statuses_count.mean())/statuses_count.std()
  statuses_count=torch.tensor(np.array(statuses_count))

  numerical_properties=torch.cat((followers_count.reshape([229580,1]),friends_count.reshape([229580,1]),favourites_count.reshape([229580,1]),statuses_count.reshape([229580,1]),screen_name_length_days.reshape([229580,1]),active_days.reshape([229580,1])),1).to(device)
  torch.save(numerical_properties,"/content/drive/MyDrive/Twibot-20/numerical_properties_tensor.pt")

else:
  numerical_properties=torch.load(root+"numerical_properties_tensor.pt").to(device)

In [ ]:
numerical_properties.size()

torch.Size([229580, 6])

In [ ]:
# categorical properties

path = root+'categorical_properties_tensor.pt'
if not os.path.isfile(path):
  category_properties=[]
  properties=['protected','geo_enabled','verified']  
  for i in range (df_data.shape[0]):
    prop=[]
    if df_data['profile'][i] is None:
      for i in range(3):
        prop.append(0)
    else:
      for each in properties:
        if df_data['profile'][i][each] is None:
          prop.append(0)
        else:
          if df_data['profile'][i][each] == "True":
            prop.append(1)
          else:
            prop.append(0)
    prop=np.array(prop)
    category_properties.append(prop)
  category_properties=torch.tensor(np.array(category_properties,dtype=np.float32)).to(device)
  torch.save(category_properties,root+'categorical_properties_tensor.pt')

else:
  category_properties=torch.load(root+"categorical_properties_tensor.pt").to(device)

In [ ]:
category_properties

tensor([[0., 1., 0.],
        [0., 0., 0.],
        [0., 1., 0.],
        ...,
        [1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
# build graph

path = root+'edge_index.pt'
if not os.path.isfile(path):
  id2index_dict={id:index for index,id in enumerate(df_data['ID'])}
  edge_index=[]
  edge_type=[]
  for i,relation in enumerate(df_data['neighbor']):
    if relation is not None:
      for each_id in relation['following']:
        try:
          target_id=id2index_dict[int(each_id)]
        except KeyError:
          continue
        else:
          edge_index.append([i,target_id])
        edge_type.append(0)
      for each_id in relation['follower']:
        try:
          target_id=id2index_dict[int(each_id)]
        except KeyError:
          continue
        else:
          edge_index.append([i,target_id])
        edge_type.append(1)
    else:
      continue
  edge_index=torch.tensor(edge_index,dtype=torch.long).t().contiguous().to(device)
  edge_type=torch.tensor(edge_type,dtype=torch.long).to(device)
  torch.save(edge_index,root+"edge_index.pt")
  torch.save(edge_type,root+"edge_type.pt")

else:
  edge_index=torch.load(root+"edge_index.pt").to(device)
  edge_type=torch.load(root+"edge_type.pt").to(device)


## Model training - common variables and functions

In [ ]:
train_idx=range(8278)
val_idx=range(8278,8278+2365)
test_idx=range(8278+2365,8278+2365+1183)


embedding_size = 128
dropout = 0.3
lr = 1e-3
weight_decay= 5e-3

epochs=100

In [ ]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.kaiming_uniform_(m.weight)

def train(epoch):
    model.train()
    output = model(description_tensor,tweets_tensor,numerical_properties,category_properties,edge_index,edge_type)
    loss_train = loss(output[train_idx], labels[train_idx])
    acc_train = accuracy(output[train_idx], labels[train_idx])
    acc_val = accuracy(output[val_idx], labels[val_idx])
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),)
    return acc_train,loss_train

def test():
    model.eval()
    output = model(description_tensor,tweets_tensor,numerical_properties,category_properties,edge_index,edge_type)
    loss_test = loss(output[test_idx], labels[test_idx])
    acc_test = accuracy(output[test_idx], labels[test_idx])
    output=output.max(1)[1].to('cpu').detach().numpy()
    label=labels.to('cpu').detach().numpy()
    f1=f1_score(label[test_idx],output[test_idx])
    mcc=matthews_corrcoef(label[test_idx], output[test_idx])
    print("Test set results:",
            "test_loss= {:.4f}".format(loss_test.item()),
            "test_accuracy= {:.4f}".format(acc_test.item()),
            "f1_score= {:.4f}".format(f1.item()),
            "mcc= {:.4f}".format(mcc.item()),
            )
    return acc_test,loss_test,f1_score

## RGCN 1 - Using all the Existing features

In [ ]:
class RGCN(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN, self).__init__()
        self.dropout = dropout
        self.linear_relu_description=nn.Sequential(
            nn.Linear(description_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet=nn.Sequential(
            nn.Linear(tweet_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_numerical_properties=nn.Sequential(
            nn.Linear(numerical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_categorical_properties=nn.Sequential(
            nn.Linear(categorical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        d=self.linear_relu_description(des)
        t=self.linear_relu_tweet(tweet)
        n=self.linear_relu_numerical_properties(numerical_properties)
        c=self.linear_relu_categorical_properties(categorical_properties)
        x=torch.cat((d,t,n,c),dim=1)
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.1065 acc_train: 0.5130 acc_val: 0.5201
Epoch: 0002 loss_train: 0.8173 acc_train: 0.6698 acc_val: 0.6596
Epoch: 0003 loss_train: 0.7959 acc_train: 0.7028 acc_val: 0.7027
Epoch: 0004 loss_train: 0.6864 acc_train: 0.7301 acc_val: 0.7214
Epoch: 0005 loss_train: 0.6304 acc_train: 0.7470 acc_val: 0.7340
Epoch: 0006 loss_train: 0.5882 acc_train: 0.7542 acc_val: 0.7514
Epoch: 0007 loss_train: 0.5785 acc_train: 0.7437 acc_val: 0.7332
Epoch: 0008 loss_train: 0.5516 acc_train: 0.7511 acc_val: 0.7353
Epoch: 0009 loss_train: 0.5526 acc_train: 0.7537 acc_val: 0.7459
Epoch: 0010 loss_train: 0.5038 acc_train: 0.7754 acc_val: 0.7674
Epoch: 0011 loss_train: 0.5037 acc_train: 0.7856 acc_val: 0.7755
Epoch: 0012 loss_train: 0.4893 acc_train: 0.7887 acc_val: 0.7814
Epoch: 0013 loss_train: 0.4973 acc_train: 0.7887 acc_val: 0.7899
Epoch: 0014 loss_train: 0.4924 acc_train: 0.7938 acc_val: 0.7852
Epoch: 0015 loss_train: 0.4826 acc_train: 0.8000 acc_val: 0.7915
Epoch: 0016 loss_train: 0

(tensor(0.8580, dtype=torch.float64),
 tensor(0.3437, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN 2 - Using only the Description feature

In [ ]:
class RGCN2(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN2, self).__init__()
        self.dropout = dropout
        self.linear_relu_description=nn.Sequential(
            nn.Linear(description_size,int(embedding_dimension)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        d=self.linear_relu_description(des)
        x=d
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN2(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.1908 acc_train: 0.4485 acc_val: 0.4693
Epoch: 0002 loss_train: 1.1601 acc_train: 0.5600 acc_val: 0.5463
Epoch: 0003 loss_train: 0.9610 acc_train: 0.5683 acc_val: 0.5539
Epoch: 0004 loss_train: 0.7698 acc_train: 0.6162 acc_val: 0.6118
Epoch: 0005 loss_train: 0.7539 acc_train: 0.6309 acc_val: 0.6376
Epoch: 0006 loss_train: 0.7594 acc_train: 0.6169 acc_val: 0.6224
Epoch: 0007 loss_train: 0.7280 acc_train: 0.5923 acc_val: 0.6030
Epoch: 0008 loss_train: 0.6979 acc_train: 0.5820 acc_val: 0.5793
Epoch: 0009 loss_train: 0.6787 acc_train: 0.5831 acc_val: 0.5945
Epoch: 0010 loss_train: 0.6767 acc_train: 0.5843 acc_val: 0.5805
Epoch: 0011 loss_train: 0.6747 acc_train: 0.5975 acc_val: 0.5877
Epoch: 0012 loss_train: 0.6725 acc_train: 0.5933 acc_val: 0.5941
Epoch: 0013 loss_train: 0.6613 acc_train: 0.6099 acc_val: 0.6042
Epoch: 0014 loss_train: 0.6470 acc_train: 0.6245 acc_val: 0.6207
Epoch: 0015 loss_train: 0.6393 acc_train: 0.6451 acc_val: 0.6529
Epoch: 0016 loss_train: 0

(tensor(0.7320, dtype=torch.float64),
 tensor(0.5685, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN 3 - Using only the Tweets feature

In [ ]:
class RGCN3(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN3, self).__init__()
        self.dropout = dropout
        self.linear_relu_tweet=nn.Sequential(
            nn.Linear(tweet_size,int(embedding_dimension)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        t=self.linear_relu_tweet(tweet)
        x=t
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN3(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 0.8508 acc_train: 0.5062 acc_val: 0.4913
Epoch: 0002 loss_train: 1.0598 acc_train: 0.5605 acc_val: 0.5531
Epoch: 0003 loss_train: 0.8289 acc_train: 0.6097 acc_val: 0.5852
Epoch: 0004 loss_train: 0.7454 acc_train: 0.6306 acc_val: 0.6233
Epoch: 0005 loss_train: 0.7058 acc_train: 0.6224 acc_val: 0.6279
Epoch: 0006 loss_train: 0.6915 acc_train: 0.6001 acc_val: 0.6085
Epoch: 0007 loss_train: 0.6743 acc_train: 0.5912 acc_val: 0.5797
Epoch: 0008 loss_train: 0.6627 acc_train: 0.5928 acc_val: 0.5979
Epoch: 0009 loss_train: 0.6625 acc_train: 0.5878 acc_val: 0.5776
Epoch: 0010 loss_train: 0.6618 acc_train: 0.5890 acc_val: 0.5751
Epoch: 0011 loss_train: 0.6525 acc_train: 0.5936 acc_val: 0.5793
Epoch: 0012 loss_train: 0.6421 acc_train: 0.6117 acc_val: 0.5979
Epoch: 0013 loss_train: 0.6326 acc_train: 0.6403 acc_val: 0.6342
Epoch: 0014 loss_train: 0.6289 acc_train: 0.6572 acc_val: 0.6545
Epoch: 0015 loss_train: 0.6261 acc_train: 0.6475 acc_val: 0.6507
Epoch: 0016 loss_train: 0

(tensor(0.7701, dtype=torch.float64),
 tensor(0.4959, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN 4 - Using the Numerical Properties feature

In [ ]:
class RGCN4(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN4, self).__init__()
        self.dropout = dropout
        self.linear_relu_numerical_properties=nn.Sequential(
            nn.Linear(numerical_properties_size,int(embedding_dimension)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        n=self.linear_relu_numerical_properties(numerical_properties)
        x=n
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN4(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.4224 acc_train: 0.4769 acc_val: 0.4613
Epoch: 0002 loss_train: 1.1911 acc_train: 0.5900 acc_val: 0.5865
Epoch: 0003 loss_train: 0.9963 acc_train: 0.6416 acc_val: 0.6393
Epoch: 0004 loss_train: 1.0088 acc_train: 0.6404 acc_val: 0.6516
Epoch: 0005 loss_train: 0.9357 acc_train: 0.6369 acc_val: 0.6364
Epoch: 0006 loss_train: 0.8490 acc_train: 0.6476 acc_val: 0.6414
Epoch: 0007 loss_train: 0.8009 acc_train: 0.6504 acc_val: 0.6469
Epoch: 0008 loss_train: 0.7811 acc_train: 0.6435 acc_val: 0.6431
Epoch: 0009 loss_train: 0.7687 acc_train: 0.6288 acc_val: 0.6338
Epoch: 0010 loss_train: 0.7514 acc_train: 0.6264 acc_val: 0.6262
Epoch: 0011 loss_train: 0.7522 acc_train: 0.6312 acc_val: 0.6220
Epoch: 0012 loss_train: 0.6986 acc_train: 0.6477 acc_val: 0.6283
Epoch: 0013 loss_train: 0.6976 acc_train: 0.6512 acc_val: 0.6414
Epoch: 0014 loss_train: 0.6864 acc_train: 0.6566 acc_val: 0.6503
Epoch: 0015 loss_train: 0.6789 acc_train: 0.6636 acc_val: 0.6478
Epoch: 0016 loss_train: 0

(tensor(0.7287, dtype=torch.float64),
 tensor(0.5451, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN5 - Using the Categorical Properties feature

In [ ]:
class RGCN5(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN5, self).__init__()
        self.dropout = dropout
        self.linear_relu_categorical_properties=nn.Sequential(
            nn.Linear(categorical_properties_size,int(embedding_dimension)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        c=self.linear_relu_categorical_properties(categorical_properties)
        x=c
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN5(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.0062 acc_train: 0.5060 acc_val: 0.5061
Epoch: 0002 loss_train: 0.6156 acc_train: 0.6660 acc_val: 0.6812
Epoch: 0003 loss_train: 0.5629 acc_train: 0.7554 acc_val: 0.7391
Epoch: 0004 loss_train: 0.5393 acc_train: 0.7868 acc_val: 0.7729
Epoch: 0005 loss_train: 0.5078 acc_train: 0.8016 acc_val: 0.7801
Epoch: 0006 loss_train: 0.4741 acc_train: 0.7995 acc_val: 0.7839
Epoch: 0007 loss_train: 0.4705 acc_train: 0.7855 acc_val: 0.7827
Epoch: 0008 loss_train: 0.4785 acc_train: 0.7536 acc_val: 0.7366
Epoch: 0009 loss_train: 0.4912 acc_train: 0.7397 acc_val: 0.7387
Epoch: 0010 loss_train: 0.4729 acc_train: 0.7617 acc_val: 0.7459
Epoch: 0011 loss_train: 0.4583 acc_train: 0.7917 acc_val: 0.7763
Epoch: 0012 loss_train: 0.4513 acc_train: 0.8029 acc_val: 0.7869
Epoch: 0013 loss_train: 0.4529 acc_train: 0.8033 acc_val: 0.7945
Epoch: 0014 loss_train: 0.4475 acc_train: 0.8060 acc_val: 0.7966
Epoch: 0015 loss_train: 0.4502 acc_train: 0.8027 acc_val: 0.7962
Epoch: 0016 loss_train: 0

(tensor(0.8157, dtype=torch.float64),
 tensor(0.4055, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN6 - Using Description Feature and Tweets Feature

In [ ]:
class RGCN6(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN6, self).__init__()
        self.dropout = dropout
        self.linear_relu_description=nn.Sequential(
            nn.Linear(description_size,int(embedding_dimension/2)),
            nn.LeakyReLU()
        )

        self.linear_relu_tweet=nn.Sequential(
            nn.Linear(tweet_size,int(embedding_dimension/2)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        d=self.linear_relu_description(des)
        t=self.linear_relu_tweet(tweet)
        x=torch.cat((d,t),dim=1)
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN6(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.1916 acc_train: 0.4121 acc_val: 0.4055
Epoch: 0002 loss_train: 0.8418 acc_train: 0.6200 acc_val: 0.6156
Epoch: 0003 loss_train: 0.8691 acc_train: 0.6316 acc_val: 0.6414
Epoch: 0004 loss_train: 0.7913 acc_train: 0.6282 acc_val: 0.6440
Epoch: 0005 loss_train: 0.7190 acc_train: 0.6139 acc_val: 0.6237
Epoch: 0006 loss_train: 0.6599 acc_train: 0.6285 acc_val: 0.6342
Epoch: 0007 loss_train: 0.6603 acc_train: 0.6098 acc_val: 0.6186
Epoch: 0008 loss_train: 0.6916 acc_train: 0.5710 acc_val: 0.5911
Epoch: 0009 loss_train: 0.6794 acc_train: 0.5840 acc_val: 0.5928
Epoch: 0010 loss_train: 0.6522 acc_train: 0.6219 acc_val: 0.6431
Epoch: 0011 loss_train: 0.6307 acc_train: 0.6596 acc_val: 0.6778
Epoch: 0012 loss_train: 0.6238 acc_train: 0.6648 acc_val: 0.6744
Epoch: 0013 loss_train: 0.6212 acc_train: 0.6684 acc_val: 0.6753
Epoch: 0014 loss_train: 0.6202 acc_train: 0.6756 acc_val: 0.6757
Epoch: 0015 loss_train: 0.6184 acc_train: 0.6747 acc_val: 0.6850
Epoch: 0016 loss_train: 0

(tensor(0.7675, dtype=torch.float64),
 tensor(0.5357, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## RGCN7 - Using Numerical Properties feature and Categorical Properties feature

In [ ]:
class RGCN7(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(RGCN7, self).__init__()
        self.dropout = dropout
        self.linear_relu_numerical_properties=nn.Sequential(
            nn.Linear(numerical_properties_size,int(embedding_dimension/2)),
            nn.LeakyReLU()
        )
        self.linear_relu_categorical_properties=nn.Sequential(
            nn.Linear(categorical_properties_size,int(embedding_dimension/2)),
            nn.LeakyReLU()
        )
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.rgcn=RGCNConv(embedding_dimension,embedding_dimension,num_relations=2)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        n=self.linear_relu_numerical_properties(numerical_properties)
        c=self.linear_relu_categorical_properties(categorical_properties)
        x=torch.cat((n,c),dim=1)
        
        x=self.linear_relu_input(x)
        x=self.rgcn(x,edge_index,edge_type)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.rgcn(x,edge_index,edge_type)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = RGCN7(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 1.3501 acc_train: 0.5063 acc_val: 0.4918
Epoch: 0002 loss_train: 0.8284 acc_train: 0.6371 acc_val: 0.6419
Epoch: 0003 loss_train: 0.7536 acc_train: 0.6963 acc_val: 0.7074
Epoch: 0004 loss_train: 0.7081 acc_train: 0.7254 acc_val: 0.7184
Epoch: 0005 loss_train: 0.6519 acc_train: 0.7354 acc_val: 0.7294
Epoch: 0006 loss_train: 0.6275 acc_train: 0.7211 acc_val: 0.7340
Epoch: 0007 loss_train: 0.6255 acc_train: 0.7230 acc_val: 0.7235
Epoch: 0008 loss_train: 0.5828 acc_train: 0.7287 acc_val: 0.7290
Epoch: 0009 loss_train: 0.5599 acc_train: 0.7583 acc_val: 0.7607
Epoch: 0010 loss_train: 0.5327 acc_train: 0.7768 acc_val: 0.7632
Epoch: 0011 loss_train: 0.5181 acc_train: 0.7853 acc_val: 0.7712
Epoch: 0012 loss_train: 0.5293 acc_train: 0.7838 acc_val: 0.7717
Epoch: 0013 loss_train: 0.5102 acc_train: 0.7900 acc_val: 0.7721
Epoch: 0014 loss_train: 0.4968 acc_train: 0.7760 acc_val: 0.7755
Epoch: 0015 loss_train: 0.4838 acc_train: 0.7723 acc_val: 0.7683
Epoch: 0016 loss_train: 0

(tensor(0.8183, dtype=torch.float64),
 tensor(0.4037, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## GCN - Replacing RGCN with GCN

In [ ]:
class GCN(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(GCN, self).__init__()
        self.dropout = dropout
        self.linear_relu_description=nn.Sequential(
            nn.Linear(description_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet=nn.Sequential(
            nn.Linear(tweet_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_numerical_properties=nn.Sequential(
            nn.Linear(numerical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_categorical_properties=nn.Sequential(
            nn.Linear(categorical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.gcn1=GCNConv(embedding_dimension,embedding_dimension)
        self.gcn2=GCNConv(embedding_dimension,embedding_dimension)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        d=self.linear_relu_description(des)
        t=self.linear_relu_tweet(tweet)
        n=self.linear_relu_numerical_properties(numerical_properties)
        c=self.linear_relu_categorical_properties(categorical_properties)
        x=torch.cat((d,t,n,c),dim=1)
        
        x=self.linear_relu_input(x)
        x=self.gcn1(x,edge_index)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.gcn2(x,edge_index)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = GCN(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 0.7143 acc_train: 0.5197 acc_val: 0.5277
Epoch: 0002 loss_train: 0.7929 acc_train: 0.5627 acc_val: 0.5510
Epoch: 0003 loss_train: 0.6672 acc_train: 0.5843 acc_val: 0.5848
Epoch: 0004 loss_train: 0.7216 acc_train: 0.4928 acc_val: 0.5002
Epoch: 0005 loss_train: 0.6859 acc_train: 0.5244 acc_val: 0.5264
Epoch: 0006 loss_train: 0.6463 acc_train: 0.6325 acc_val: 0.6317
Epoch: 0007 loss_train: 0.6594 acc_train: 0.6123 acc_val: 0.6030
Epoch: 0008 loss_train: 0.6653 acc_train: 0.6064 acc_val: 0.5924
Epoch: 0009 loss_train: 0.6436 acc_train: 0.6434 acc_val: 0.6288
Epoch: 0010 loss_train: 0.6268 acc_train: 0.6647 acc_val: 0.6638
Epoch: 0011 loss_train: 0.6283 acc_train: 0.6406 acc_val: 0.6368
Epoch: 0012 loss_train: 0.6374 acc_train: 0.6219 acc_val: 0.6279
Epoch: 0013 loss_train: 0.6287 acc_train: 0.6359 acc_val: 0.6359
Epoch: 0014 loss_train: 0.6208 acc_train: 0.6544 acc_val: 0.6668
Epoch: 0015 loss_train: 0.6121 acc_train: 0.6781 acc_val: 0.6753
Epoch: 0016 loss_train: 0

(tensor(0.7430, dtype=torch.float64),
 tensor(0.5333, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## GAT (Graph Attention Networks) - replace RGCN with GAT

In [ ]:
class GAT(nn.Module):
    def __init__(self,description_size=768,tweet_size=768,numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128,dropout=0.3):
        super(GAT, self).__init__()
        self.dropout = dropout
        self.linear_relu_descriptioncription=nn.Sequential(
            nn.Linear(description_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_tweet=nn.Sequential(
            nn.Linear(tweet_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_numerical_properties=nn.Sequential(
            nn.Linear(numerical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        self.linear_relu_categorical_properties=nn.Sequential(
            nn.Linear(categorical_properties_size,int(embedding_dimension/4)),
            nn.LeakyReLU()
        )
        
        self.linear_relu_input=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_relu_output1=nn.Sequential(
            nn.Linear(embedding_dimension,embedding_dimension),
            nn.LeakyReLU()
        )
        self.linear_output2=nn.Linear(embedding_dimension,2)
        
        self.gat1=GATConv(embedding_dimension,int(embedding_dimension/4),heads=4)
        self.gat2=GATConv(embedding_dimension,embedding_dimension)
        
    def forward(self,des,tweet,numerical_properties,categorical_properties,edge_index,edge_type):
        d=self.linear_relu_descriptioncription(des)
        t=self.linear_relu_tweet(tweet)
        n=self.linear_relu_numerical_properties(numerical_properties)
        c=self.linear_relu_categorical_properties(categorical_properties)
        x=torch.cat((d,t,n,c),dim=1)
        
        x=self.linear_relu_input(x)
        x=self.gat1(x,edge_index)
        x=F.dropout(x,p=self.dropout,training=self.training)
        x=self.gat2(x,edge_index)
        x=self.linear_relu_output1(x)
        x=self.linear_output2(x)
            
        return x

In [ ]:
model = GAT(numerical_properties_size=6,categorical_properties_size=3,embedding_dimension=128).to(device)
loss=nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,weight_decay=weight_decay)

model.apply(init_weights)

for epoch in range(epochs):
    train(epoch)
    
test()

Epoch: 0001 loss_train: 0.8150 acc_train: 0.4569 acc_val: 0.4609
Epoch: 0002 loss_train: 0.8101 acc_train: 0.5633 acc_val: 0.5526
Epoch: 0003 loss_train: 0.6796 acc_train: 0.5641 acc_val: 0.5543
Epoch: 0004 loss_train: 0.7297 acc_train: 0.5011 acc_val: 0.5150
Epoch: 0005 loss_train: 0.6854 acc_train: 0.5737 acc_val: 0.5510
Epoch: 0006 loss_train: 0.6466 acc_train: 0.6223 acc_val: 0.6182
Epoch: 0007 loss_train: 0.6585 acc_train: 0.6034 acc_val: 0.5915
Epoch: 0008 loss_train: 0.6640 acc_train: 0.6091 acc_val: 0.5894
Epoch: 0009 loss_train: 0.6395 acc_train: 0.6212 acc_val: 0.6199
Epoch: 0010 loss_train: 0.6239 acc_train: 0.6508 acc_val: 0.6423
Epoch: 0011 loss_train: 0.6279 acc_train: 0.6698 acc_val: 0.6643
Epoch: 0012 loss_train: 0.6329 acc_train: 0.6639 acc_val: 0.6588
Epoch: 0013 loss_train: 0.6217 acc_train: 0.6737 acc_val: 0.6584
Epoch: 0014 loss_train: 0.6074 acc_train: 0.6793 acc_val: 0.6702
Epoch: 0015 loss_train: 0.6019 acc_train: 0.6625 acc_val: 0.6571
Epoch: 0016 loss_train: 0

(tensor(0.7777, dtype=torch.float64),
 tensor(0.4917, grad_fn=<NllLossBackward0>),
 <function sklearn.metrics._classification.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>)

## Graphs & Results

In [ ]:
import plotly.express as px

In [ ]:
df = pd.read_csv(root+'results.csv')

df

,Method,loss_train,acc_train,acc_val,test_loss,test_accuracy,f1_score,mcc
0,RGCN,0.2420,0.8973,0.8567,0.3437,0.8580,0.8721,0.7139
1,RGCN2,0.4194,0.8095,0.7383,0.5685,0.7320,0.7681,0.4590
2,RGCN3,0.4382,0.7987,0.7810,0.4959,0.7701,0.7824,0.5397
3,RGCN4,0.5255,0.7333,0.7108,0.5451,0.7287,0.7775,0.4598
4,RGCN5,0.4214,0.8088,0.7928,0.4055,0.8157,0.8541,0.6663
5,RGCN6,0.3403,0.8564,0.7886,0.5357,0.7675,0.7850,0.5320
6,RGCN7,0.3990,0.8166,0.7928,0.4037,0.8183,0.8538,0.6621
7,GCN,0.4502,0.7809,0.7395,0.5333,0.7430,0.7606,0.4834
8,GAT,0.4247,0.8050,0.7700,0.4917,0.7777,0.8053,0.5520


In [ ]:
# test accuracy bar graph

fig = px.bar(df, x='Method', y='test_accuracy',
             hover_data=['acc_train', 'f1_score', 'mcc'], color='test_accuracy',
             labels={'Test Accuracy'}, height=400)
fig.show()

In [ ]:
# MCC bar graph
fig = px.bar(df, y='mcc', x='Method', text_auto='.4f',
            title="MCC")
fig.show()

In [ ]:
# f1 score bar graph
fig = px.bar(df, y='f1_score', x='Method', text_auto='.4f',
            title="f1-score")
fig.show()